# absorption_incomplete
This notebook could be useful later to determine the theoretical effects of absorption on detection, following what was discussed by Dent et.al. Currently it creates a set of lattice points but can't do much else.  

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import astropy.units as unit
import astropy.constants as const
import itertools
import plotly.express as px 
import pandas as pd
from scipy.special import factorial as fact
from scipy.optimize import root_scalar

from axion_functions import *

In [5]:
#Define constants
a_nai = 6.642 #Lattice constant, Angstroms = 1e-10m, from Dent paper. 
hbar_kevs = (const.hbar.to(unit.keV*unit.s)).value #hbar, keV*s, from astropy
c_aas = (const.c.to(unit.AA/unit.s)).value #speed of light, angstrom/s (cancels with a), from astropy

mass = 50*unit.kg
vol = ((mass/(3.67*unit.g*(unit.cm)**(-3))).to(unit.AA**3)).value #Crystal volume, in AA^3

vol_cell = 67.71 #In AA^3, from Dent paper

s_in_y = 365.25*24*60*60 #Number of seconds in a year

In [11]:
def lattice_3d_scratch(N_x, N_y, N_z):
    full_grid = np.array(tuple(itertools.product(np.arange(0,N_x+0.5,0.5),
                                                 np.arange(0,N_y+0.5,0.5),
                                                 np.arange(0,N_z+0.5,0.5))))
    return np.array(list(filter(lambda x: sum(x%1)%1 == 0, full_grid)))

d3_lattice = lattice_3d_scratch(5,
                                5,
                                5)
#Takes 1m13s for 100^3

In [12]:
fig = px.scatter_3d(x = d3_lattice[:,0],
                    y = d3_lattice[:,1],
                    z = d3_lattice[:,2])



fig.update_traces(marker_size = 5)
fig.show()


In [13]:
Na_lattice = lattice_3d_scratch(1,1,1)
I_lattice  = Na_lattice + np.array([1/2,1/2,1/2])


fig = go.Figure()

scatter = fig.add_trace(go.Scatter3d(
    x=Na_lattice[:,0],
    y=Na_lattice[:,1],
    z=Na_lattice[:,2],
    mode='markers',
    name = "Na",
    marker = dict(
        size=5,
        color='blue'
    )
))

scatter_io = fig.add_trace(go.Scatter3d(
    x=I_lattice[:,0],
    y=I_lattice[:,1],
    z=I_lattice[:,2],
    name = "I",
    mode='markers',
    marker = dict(
        size=5,
        color='red'
    )
))

fig.show()

In [ ]:
def difference_array(lattice):
    output = []
    for vec1 in lattice:
        for vec2 in lattice:
            output.append(vec1 - vec2)
    return np.array(output)
#This will take like 28 Australian election cycles to run as is
#This essentially forms an antisymmetric tensor: treating it as such could cut down time
#JIT compilation w/ numba ould also help